In [1]:
import sys
sys.path.append('..')

import numpy as np 
import pandas as pd 

from mod.SimSale import SimSale
from mod.Sale import Sale
from mod.Demand import Demand

# from numdifftools import Hessian,Jacobian

## 1.模拟数据 及 待估计参数

In [2]:
sim_sale=SimSale(level_size=[2,2],seed=1)
sale_df = sim_sale.generate_sale(lam=1000,size=100,un_ava_frac=0.5,un_ava_mix=True)
print(sim_sale.attr_f_list[0])
print(sim_sale.attr_f_list[1])
print(sim_sale.attr_pi_list[0])
print(sim_sale.attr_pi_list[1]) 
mod = Demand(data=sale_df)
sale_df

a    0.350015
b    0.649985
Name: A, dtype: float64
a    0.669174
b    0.330826
Name: B, dtype: float64
          a         b
a  1.000000  0.853785
b  0.280648  1.000000
          a         b
a  1.000000  0.024803
b  0.678162  1.000000


,A_a*B_a,A_a*B_b,A_b*B_a,A_b*B_b
0,350.0,NaN,NaN,308.0
1,352.0,NaN,NaN,346.0
2,NaN,NaN,627.0,323.0
3,372.0,NaN,NaN,317.0
4,NaN,NaN,594.0,319.0
...,...,...,...,...
95,368.0,174.0,NaN,NaN
96,373.0,182.0,NaN,NaN
97,NaN,NaN,617.0,325.0
98,NaN,113.0,805.0,NaN


## 2.参数估计

In [ ]:
# method='dual_annealing' / 'differential_evolution'
mod.fit(method='dual_annealing',maxiter=1000)
# mod.fit(method='differential_evolution')

mod.summary(SimSale=sim_sale)

In [ ]:
df = sale_df.iloc[[1],:]
print(df.sum(axis=1))

mod.predict(df).sum(axis=1)

In [ ]:
df.isna().any(axis=1).to_numpy()[0]

In [3]:
sale = Sale(attrs_num=[3,3])
sale.sim(size=100,unsale_pct=0.25)
df = sale.data_observed
df

,A_a*B_a,A_a*B_b,A_a*B_c,A_b*B_a,A_b*B_b,A_b*B_c,A_c*B_a,A_c*B_b,A_c*B_c
0,168,64,75,242,146,196,84,NaN,NaN
1,135,81,101,233,164,170,93,NaN,NaN
2,133,84,97,230,161,165,109,NaN,NaN
3,136,77,82,234,175,182,91,NaN,NaN
4,149,83,85,226,158,190,87,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,156,76,94,230,147,172,104,NaN,NaN
96,128,83,83,242,150,191,103,NaN,NaN
97,152,68,82,206,183,180,105,NaN,NaN
98,160,69,86,215,152,176,119,NaN,NaN


In [4]:
mod = Demand(data=df)
mod.fit(method='dual_annealing',maxiter=1000)

In [5]:
mod.summary()

#################### 1.属性的选择概率 ####################
     Estimate     Guess   EG_Diff
A_a  0.305769  0.314740 -0.008971
A_b  0.482091  0.582603 -0.100513
A_c  0.212140  0.102656  0.109484
     Estimate     Guess   EG_Diff
B_a  0.470119  0.483912 -0.013792
B_b  0.250065  0.243554  0.006512
B_c  0.279815  0.272534  0.007281


In [6]:
mod.predict(df.iloc[[1]])

,A_a*B_a,A_a*B_b,A_a*B_c,A_b*B_a,A_b*B_b,A_b*B_c,A_c*B_a,A_c*B_b,A_c*B_c
0,146,78,87,230,122,137,101,54,60


In [7]:
sale.data_origin.iloc[[1]]

,A_a*B_a,A_a*B_b,A_a*B_c,A_b*B_a,A_b*B_b,A_b*B_c,A_c*B_a,A_c*B_b,A_c*B_c
1,135,81,101,233,126,120,93,48,63


,A_a*B_a,A_a*B_b,A_a*B_c,A_b*B_a,A_b*B_b,A_b*B_c,A_c*B_a,A_c*B_b,A_c*B_c
0,168,64,75,242,146,196,84,NaN,NaN
1,135,81,101,233,164,170,93,NaN,NaN
2,133,84,97,230,161,165,109,NaN,NaN
3,136,77,82,234,175,182,91,NaN,NaN
4,149,83,85,226,158,190,87,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,156,76,94,230,147,172,104,NaN,NaN
96,128,83,83,242,150,191,103,NaN,NaN
97,152,68,82,206,183,180,105,NaN,NaN
98,160,69,86,215,152,176,119,NaN,NaN


In [6]:
data_sr = df.iloc[[1]].squeeze()
index_unsale = data_sr.isna().to_list()
index_sale   = data_sr.notna().to_list()
goods_f_unsale = sale.theta_goods_f[index_unsale]
goods_f_sale = sale.theta_goods_f[index_sale]
goods_pi_unsale_to_sale = sale.theta_goods_pi[index_unsale][:,index_sale]
goods_pi_unsale_to_sale = mod.get_trans_prob(goods_pi_unsale_to_sale)
total_sale = data_sr.to_numpy()[index_sale] / (goods_f_sale + np.dot(goods_f_unsale , goods_pi_unsale_to_sale))



In [9]:
sale.get_trans_prob(goods_pi_unsale_to_sale)

array([[0.       , 0.       , 0.       , 0.       , 0.7884287, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.7884287,
        0.       ]])

In [11]:
total_sale

array([ 928.28032655, 1052.92378917, 1174.38807039, 1033.24061121,
       1016.87407833,  942.86733039,  923.63824929])

In [40]:
total_sale = data_sr.to_numpy()[index_sale] / (goods_f_sale + goods_f_unsale * goods_pi_unsale_to_sale)

ValueError: operands could not be broadcast together with shapes (7,) (7,2) 

In [46]:
(goods_f_sale.reshape(-1,1) + goods_f_unsale * goods_pi_unsale_to_sale)

array([[0.14845987, 0.14698449],
       [0.08403292, 0.07847887],
       [0.08721414, 0.09394445],
       [0.2314552 , 0.22855715],
       [0.13324034, 0.12233064],
       [0.13573559, 0.14895579],
       [0.1234025 , 0.11234144]])

In [45]:
goods_f_sale

array([0.14543021, 0.07692864, 0.08600224, 0.22550411, 0.11928557,
       0.13335508, 0.10068877])